In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#### Read in Datasets and Files

* railOD_tt_fare contains the travel times, distances and fares provided by WMATA
* stndists contains the OD pair distances via OSMNX
* mstns contains the station ID and two formattings of the station name 

In [2]:
railOD_tt_fare = pd.read_excel("../../Data/railOD_TravelTimesAndFares.xlsx")
track_miles = pd.read_excel("../../Data/railOD_trackMiles_spring2022.xlsx", header=1, index_col=0)
mstns2 = pd.read_excel("../../Data/mstn_id_to_stn_name.xlsx")

#### Data Preprocessing

* Create a column of Station names that can easily be merged with track miles
* Convert track miles into OD pair format and create an od pair identifying column for merging using the MSTN station ID codes
* Create an od pair identifying column for merging using the MSTN station ID codes for the railOD_tt_fare dataset
* Merge the track_miles and railOD_peak_fare and calculate fare per track_mile
* Merge the track_miles and railOD_off_peak_fare and calculate fare per track_mile
* Export both the peak and off peak fares per mile

In [3]:
mstns2.sort_values(by="PRIMARY_NAME_FY23", inplace=True)
mstns2['track_miles_names'] = ['Addison Road', 'Anacostia', 'Archives', 'Arlington Cemetery', "",
       'Ballston-MU', 'Benning Road', 'Bethesda', 'Braddock Road',
       'Branch Ave', 'Brookland-CUA', 'Capitol Heights', 'Capitol South',
       'Cheverly', 'Clarendon', 'Cleveland Park', 'College Park-U of Md',
       'Columbia Heights', 'Congress Heights', 'Court House',
       'Crystal City', 'Deanwood', 'Largo Town Center', "", 'Dunn Loring', 'Dupont Circle',
       'East Falls Church', 'Eastern Market', 'Eisenhower Ave',
       'Farragut North', 'Farragut West', 'Federal Center SW',
       'Federal Triangle', 'Foggy Bottom-GWU', 'Forest Glen',
       'Fort Totten', 'Franconia-Springfield', 'Friendship Heights',
       'Gallery Place', 'Georgia Ave-Petworth', 'Glenmont', 'Greenbelt',
       'Greensboro', 'Grosvenor-Strathmore', "",'Huntington',"Prince George's Plaza", "",
       'Judiciary Square', 'King St-Old Town', "L'Enfant Plaza",
       'Landover',"", 'McLean', 'McPherson Square',
       'Medical Center', 'Metro Center', 'Minnesota Ave',
       'Morgan Boulevard', 'Mt Vernon Sq', 'Navy Yard-Ballpark',
       'Naylor Road', 'New Carrollton', 'NoMa-Gallaudet U','White Flint', 'Pentagon',
       'Pentagon City', 'Potomac Ave', "","",
       'Rhode Island Ave', 'Rockville',
       'Ronald Reagan Washington National Airport', 'Rosslyn',
       'Shady Grove', 'Shaw-Howard Univ', 'Silver Spring', 'Smithsonian',
       'Southern Ave', 'Spring Hill', 'Stadium-Armory', 'Suitland',
       'Takoma', 'Tenleytown-AU', 'Twinbrook', 'Tysons Corner',
       'U Street', 'Union Station', 'Van Dorn Street', 'Van Ness-UDC',
       'Vienna', 'Virginia Square-GMU', 'Waterfront', 'West Falls Church',
       'West Hyattsville', 'Wheaton',  'Wiehle',
       'Woodley Park']

In [4]:

track_miles = track_miles.stack().rename_axis(('O', 'D')).reset_index(name="track_miles")
track_miles.head()

,O,D,track_miles
0,Addison Road,Addison Road,0.10
1,Addison Road,Anacostia,10.35
2,Addison Road,Archives,8.31
3,Addison Road,Arlington Cemetery,11.39
4,Addison Road,Ballston-MU,14.27


In [5]:

track_miles2 = mstns2.merge(track_miles, left_on ='track_miles_names' , right_on= 'O')
track_miles2 = mstns2.merge(track_miles2, left_on ='track_miles_names' , right_on='D' )

In [6]:
track_miles2.drop(["PRIMARY_NAME_FY23_x", "PRIMARY_NAME_FY23_y", "track_miles_names_x", "track_miles_names_y"], axis=1, inplace=True)
track_miles2['pairs'] = track_miles2.apply(lambda x: str(x['ID_y'])+str(0)+str(x['ID_x']), axis=1)
track_miles2['ID1'] = track_miles2['ID_y'].replace('[A-Z]{4}_0', '', regex=True)
track_miles2['ID2'] = track_miles2['ID_x'].replace('[A-Z]{4}_0', '', regex=True)
track_miles2['ID1'] = track_miles2['ID1'].str.lstrip("0")
track_miles2['ID2'] = track_miles2['ID2'].str.lstrip("0")
track_miles2['pairs2'] = track_miles2.apply(lambda x: str(x['ID1'])+str(0)+str(x['ID2']), axis=1)
track_miles2.head()

,ID_x,ID_y,O,D,track_miles,pairs,ID1,ID2,pairs2
0,MSTN_062,MSTN_062,Addison Road,Addison Road,0.10,MSTN_0620MSTN_062,62,62,62062
1,MSTN_062,MSTN_001,Anacostia,Addison Road,10.35,MSTN_0010MSTN_062,1,62,1062
2,MSTN_062,MSTN_002,Archives,Addison Road,8.31,MSTN_0020MSTN_062,2,62,2062
3,MSTN_062,MSTN_065,Arlington Cemetery,Addison Road,11.39,MSTN_0650MSTN_062,65,62,65062
4,MSTN_062,MSTN_068,Ballston-MU,Addison Road,14.27,MSTN_0680MSTN_062,68,62,68062


In [7]:
#create pairs
railOD_tt_fare['mstn_id_o'] = railOD_tt_fare['O_MSTN_ID'].str[-2:].str.lstrip('0')
railOD_tt_fare['mstn_id_d'] = railOD_tt_fare['D_MSTN_ID'].str[-2:].str.lstrip('0')
railOD_tt_fare['pairs'] = railOD_tt_fare['mstn_id_o']+str(0)+railOD_tt_fare['mstn_id_d']
railOD_tt_fare.head()

,O_MSTN_ID,D_MSTN_ID,O_PRIMARY_NAME,D_PRIMARY_NAME,COMP_MILE,PEAK_FARE,OFF_PEAK_FARE,SD_FARE,TRAVEL_TIME,mstn_id_o,mstn_id_d,pairs
0,MSTN_001,MSTN_001,Anacostia,Anacostia,0.05,2.25,2.0,1.10,0,1,1,101
1,MSTN_001,MSTN_002,Anacostia,Archives,2.83,2.25,2.0,1.10,9,1,2,102
2,MSTN_001,MSTN_003,Anacostia,Benning Road,5.76,3.15,2.6,1.55,26,1,3,103
3,MSTN_001,MSTN_004,Anacostia,Brookland-CUA,5.98,3.20,2.6,1.60,27,1,4,104
4,MSTN_001,MSTN_005,Anacostia,Capitol South,2.55,2.25,2.0,1.10,16,1,5,105


In [8]:
railOD_peak_fare = railOD_tt_fare[['O_MSTN_ID', 'D_MSTN_ID', 'pairs', 'PEAK_FARE']]
railOD_off_peak_fare = railOD_tt_fare[['O_MSTN_ID', 'D_MSTN_ID','pairs', 'OFF_PEAK_FARE']]

In [9]:
railOD_peak_fare_per_mile2 = railOD_peak_fare.merge(track_miles2, left_on=['pairs'], right_on=['pairs2'])
railOD_peak_fare_per_mile2['peak_fare_per_mile2'] = railOD_peak_fare_per_mile2['PEAK_FARE']/railOD_peak_fare_per_mile2['track_miles']
railOD_peak_fare_per_mile2 = railOD_peak_fare_per_mile2[['pairs2', 'O_MSTN_ID', 'D_MSTN_ID', 'peak_fare_per_mile2', 'PEAK_FARE']]

In [10]:
railOD_peak_fare_per_mile2.head()

,pairs2,O_MSTN_ID,D_MSTN_ID,peak_fare_per_mile2,PEAK_FARE
0,101,MSTN_001,MSTN_001,22.500000,2.25
1,102,MSTN_001,MSTN_002,0.712025,2.25
2,103,MSTN_001,MSTN_003,0.399240,3.15
3,104,MSTN_001,MSTN_004,0.446927,3.20
4,105,MSTN_001,MSTN_005,0.641026,2.25


In [11]:
railOD_off_peak_fare_per_mile2 = railOD_off_peak_fare.merge(track_miles2, left_on=['pairs'], right_on=['pairs2'])
railOD_off_peak_fare_per_mile2['off_peak_fare_per_mile'] = railOD_off_peak_fare_per_mile2['OFF_PEAK_FARE']/railOD_off_peak_fare_per_mile2['track_miles']
railOD_off_peak_fare_per_mile2 = railOD_off_peak_fare_per_mile2[['pairs2', 'O_MSTN_ID', 'D_MSTN_ID', 'off_peak_fare_per_mile', 'OFF_PEAK_FARE']]

In [12]:
railOD_off_peak_fare_per_mile2.head()

,pairs2,O_MSTN_ID,D_MSTN_ID,off_peak_fare_per_mile,OFF_PEAK_FARE
0,101,MSTN_001,MSTN_001,20.000000,2.0
1,102,MSTN_001,MSTN_002,0.632911,2.0
2,103,MSTN_001,MSTN_003,0.329531,2.6
3,104,MSTN_001,MSTN_004,0.363128,2.6
4,105,MSTN_001,MSTN_005,0.569801,2.0


In [13]:
railOD_off_peak_fare_per_mile2.to_excel("output/railOD_off_peak_fare_per_mile.xlsx", sheet_name='railOD_off_peak_fare_per_mile', index=True)

In [14]:
railOD_peak_fare_per_mile2.to_excel("output/railOD_peak_fare_per_mile.xlsx", sheet_name='railOD_peak_fare_per_mile', index=True)